In [1]:
import urllib.request, urllib.error
import pandas as pd
import json
import csv
import os
import time
import re

In [2]:
# Search keywords
# vaccine passport
# covid passport
# vaccine certificate

In [3]:
def crawl_data(subreddit, search_param):
    iteration = 0

    # time in epoch. 
    # used for crawling to iterate in time (epoch value)
    # 86400 in epochs --> 1 (one) day.
    search_iteration_interval = 86400

    # performs full-text search (based on elasticsearch)
    # for before, after and q params visit: https://github.com/pushshift/api#search-parameters-for-comments
    seach_param = search_param
    subreddit = subreddit
    filename = str(subreddit + "_" + seach_param + "_" + 'comments.csv')

    before = int(time.time())
    after = before - search_iteration_interval

    seach_param = re.sub(" ", "%20", seach_param)

    # as one iteration searches for one-day comments, this will search comments from now on to the past 400 days.
    while iteration < 400:

        url = "https://api.pushshift.io/reddit/search/comment/?q="+ str(seach_param) +"&subreddit=" +str(subreddit)+ "&after=" + str(after) + "&before=" + str(before) + "&sort=asc&size=100"

        try:
            data = urllib.request.urlopen(url).read()

        except urllib.error.HTTPError as e:
            print('HTTPError: {}'.format(e.code))

        except urllib.error.URLError as e:
            print('URLError: {}'.format(e.reason))

        else:
            # 200 - success.
            # print('200 - success.')
            # print(url)
            data = json.loads(data)

            df = pd.DataFrame(data['data'])

            if 'created_utc' in df.columns:
                df = df[['created_utc', 'body', 'subreddit', 'subreddit_id', 'permalink']]

                # en eski tarih
                #after = df['created_utc'].iloc[-1] - 432.000
                after = df['created_utc'].iloc[0] - search_iteration_interval

                # en guncel tarih en altta.
                #before = df['created_utc'].iloc[-1]
                before = df['created_utc'].iloc[0]

                df.to_csv(filename, mode='a', header=not os.path.exists(filename))

                iteration = iteration + 1
                #print(iteration)
                time.sleep(2)

            else:  
                iteration = iteration + 1
                #print('no comment found regarding the search param (q) in iteration ', iteration)
                before = before - search_iteration_interval
                after = after - search_iteration_interval
                time.sleep(2)
                continue
    print('Comments saved to file: ', filename)

In [4]:
subreddits = ['Nigeria', 'pakistan', 'ukraine', 'Philippines', 'VietNam']
keywords = ['vaccine passport', 'covid passport', 'vaccine certificate']

In [5]:
for subr in subreddits:
    for keyw in keywords:
        crawl_data(subr, keyw)

HTTPError: 502
Comments saved to file:  Nigeria_vaccine passport_comments.csv
Comments saved to file:  Nigeria_covid passport_comments.csv
HTTPError: 502
Comments saved to file:  Nigeria_vaccine certificate_comments.csv
Comments saved to file:  pakistan_vaccine passport_comments.csv
HTTPError: 502
HTTPError: 502
HTTPError: 502
HTTPError: 502
HTTPError: 502
Comments saved to file:  pakistan_covid passport_comments.csv
HTTPError: 502
HTTPError: 502
Comments saved to file:  pakistan_vaccine certificate_comments.csv
Comments saved to file:  ukraine_vaccine passport_comments.csv
Comments saved to file:  ukraine_covid passport_comments.csv
Comments saved to file:  ukraine_vaccine certificate_comments.csv
Comments saved to file:  Philippines_vaccine passport_comments.csv
Comments saved to file:  Philippines_covid passport_comments.csv
Comments saved to file:  Philippines_vaccine certificate_comments.csv
Comments saved to file:  VietNam_vaccine passport_comments.csv
Comments saved to file:  Vi